In [15]:
# 리밸런싱 
# out-of-sample에서 리밸런싱 우선
# 매달 리밸런싱 진행
    # 리밸런싱은 해당 기간 최적화 돌려서 나온 비중을 갖고 '매달' 진행.
    # 1. (out of sample 기간) 2023.05.01에 투자를 시작할 때 과거 5년치 데이터로 최적화 문제 풀고, 다음 한달간 수익률 보기. 
    # 그리고 2023.06.01에 시작할 때는 그때까지 데이터로 최적화 풀고, 
    # 나온 가중치로 리밸런싱 하고 한달간 수익률 보기… 
    # 반복
# 2. look back 기간을 1년, 2년, 등 여러 기간으로 설정하고 돌려보기.

In [16]:
# !pip install pyPortfolioOpt
# !pip install cplex
# !pip install numpy
# !pip install pandas
# !pip install cvxpy

In [17]:
import numpy as np
import cvxpy as cp
import cplex
from tqdm import tqdm
from datetime import timedelta

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader.data as pdr
import yfinance as yf

from dateutil.relativedelta import relativedelta
from datetime import datetime

In [18]:
# GPT 종목 (GPT weight)
stocks_15 = {
    "AAPL": 0.084, "AMZN": 0.074, "NVDA": 0.064, "JPM": 0.054, "PG": 0.054,
    "PFE": 0.064, "JNJ": 0.064, "KO": 0.054, "XOM": 0.064, "NEE": 0.074,
    "GOOGL": 0.084, "MSFT": 0.084, "TSLA": 0.074, "NKE": 0.054, "BAC": 0.054
}

stocks_30 = {
    "MSFT": 0.067, "AMZN": 0.067, "NVDA": 0.067, "AAPL": 0.067, "GOOGL": 0.067,
    "ADBE": 0.067, "JNJ": 0.05, "PFE": 0.05, "MRK": 0.05, "ABT": 0.05, 
    "PG": 0.05, "KO": 0.05, "JPM": 0.05, "GS": 0.05, "CAT": 0.025, 
    "CVX": 0.025, "XOM": 0.025, "BA": 0.025, "TSLA": 0.025, "NEE": 0.025, 
    "NKE": 0.005, "VZ": 0.005, "CRM": 0.005, "UNH": 0.005, "WMT": 0.005, 
    "QCOM": 0.005, "BAC": 0.005, "V": 0.005, "MCD": 0.005, "INTC": 0.005
}

stocks_45 = {
    "AAPL": 0.05, "GOOGL": 0.04, "MSFT": 0.04, "NVDA": 0.03, "AMD": 0.03, 
    "ORCL": 0.02, "CRM": 0.02, "INTC": 0.01, "CSCO": 0.01, "JPM": 0.04, 
    "GS": 0.03, "BAC": 0.03, "MS": 0.02, "AXP": 0.02, "C": 0.01,
    "JNJ": 0.03, "UNH": 0.03, "PFE": 0.02, "ABBV": 0.02, "TSLA": 0.02, 
    "AMGN": 0.02, "GILD": 0.01, "PG": 0.03, "KO": 0.03, "NKE": 0.02, 
    "PEP": 0.02, "COST": 0.02, "WMT": 0.02, "TGT": 0.01, "XOM": 0.025, 
    "CVX": 0.025, "NEE": 0.02, "DUK": 0.01, "SO": 0.01, "SLB": 0.01,
    "MMM": 0.02, "CAT": 0.02, "HON": 0.02, "GE": 0.02, "ADP": 0.02,
    "AMZN": 0.02, "META": 0.02, "HD": 0.02, "VZ": 0.01, "MRK": 0.01
}

In [19]:
# 종목명 리스트
tickers_15 = list(stocks_15.keys())
tickers_30 = list(stocks_30.keys())
tickers_45 = list(stocks_45.keys())

## GPT weight

In [20]:
gpt_15_weights = stocks_15
gpt_30_weights = stocks_30
gpt_45_weights = stocks_45

## Equal weight

In [21]:
eq_15_weights = [1/15] * 15
eq_30_weights = [1/30] * 30
eq_45_weights = [1/45] * 45

eq_15 =dict(zip(tickers_15, eq_15_weights))
eq_30 =dict(zip(tickers_30, eq_30_weights))
eq_45 =dict(zip(tickers_45, eq_45_weights))

## GMV + EPO weight

In [22]:
gmv_epo_15_weights = [0.049605485006088634, 0.07849495425750663, 0.06333929068001071, 0.05700662407849795, 0.07650289062642221, 0.05900628444525451, 0.057323863755786, 0.0475050526473884, 0.06851785110258568, 0.0551095984517666, 0.0635746953436858, 0.08684669491598282, 0.06555603211482636, 0.08160761317658853, 0.090003069397609]
gmv_epo_30_weights = [0.016666779461997225, 0.028582382826203727, 0.03181481366190954, 0.04612454337672658, 0.03430208600932986, 0.02496457702197057, 0.034559689579773685, 0.03603933972602465, 0.0348256115466567, 0.02719368909866466, 0.01666666768426058, 0.03550281897656019, 0.03316982644195968, 0.02106295226563651, 0.019372147251594442, 0.01873074735009172, 0.06623366706426345, 0.016666690262465863, 0.03423317149469612, 0.02304327143120648, 0.031214508772612753, 0.045807643648171685, 0.027946448832753296, 0.049668167222889026, 0.049588148769328606, 0.024769224302769302, 0.016666667674594095, 0.05283625719167181, 0.05583979037764232, 0.045907670675591185]
gmv_epo_45_weights = [0.011111111975729409, 0.026646602204962858, 0.011111111299922065, 0.0309210193238956, 0.025007258123960922, 0.034659892275724705, 0.01111111146901072, 0.011111111747740477, 0.011111111516746397, 0.01779091912009698, 0.01984888218960698, 0.02522647335013622, 0.011180137115527101, 0.017489649124790285, 0.028191045792364738, 0.03631158995254371, 0.044444444090877805, 0.016302194182152667, 0.011111111266490585, 0.011111111595258412, 0.01111111125737019, 0.022827040798679366, 0.020587601244674014, 0.011111111503325005, 0.011111111556128327, 0.044444443639370106, 0.011111111461049875, 0.04444444402780661, 0.011111111368812045, 0.011111112285545484, 0.023035029376855205, 0.01260693000477375, 0.02165124524507929, 0.02165119066334294, 0.011286898608143595, 0.029528803525066108, 0.017506358987491956, 0.03972350417693582, 0.018956379525549516, 0.036275899230639834, 0.040688577981395774, 0.011896549529971394, 0.03665347882393091, 0.03968317677255412, 0.028087890687943526]

gmv_epo_15 = dict(zip(tickers_15, gmv_epo_15_weights))
gmv_epo_30 = dict(zip(tickers_30, gmv_epo_30_weights))
gmv_epo_45 = dict(zip(tickers_45, gmv_epo_45_weights))

## MAX Sharpe + EPO weight

In [23]:
msp_epo_15_weights = [0.13333, 0.03333, 0.13333, 0.03333, 0.09539, 0.03333, 0.03333, 0.03333, 0.03707, 0.05699, 0.03897, 0.13333, 0.13333, 0.03825, 0.03333]
msp_epo_30_weights = [0.06667, 0.01667, 0.06667, 0.06667, 0.0387, 0.01667, 0.01667, 0.01667, 0.06667, 0.02469, 0.06667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667, 0.06667, 0.0443, 0.02974, 0.01667, 0.01667, 0.05967, 0.04331, 0.06667, 0.01667, 0.01667, 0.02625, 0.01667]
msp_epo_45_weights = [0.04444, 0.04379, 0.04444, 0.04444, 0.04444, 0.04444, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.04444, 0.01111, 0.01111, 0.04444, 0.01111, 0.01111, 0.04444, 0.01111, 0.0312, 0.02923, 0.04444, 0.03726, 0.04444, 0.01557, 0.01111, 0.04296, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.04444]

msp_epo_15 = dict(zip(tickers_15, msp_epo_15_weights))
msp_epo_30 = dict(zip(tickers_30, msp_epo_30_weights))
msp_epo_45 = dict(zip(tickers_45, msp_epo_45_weights))

## MAX Returns + EPO weight

In [24]:
max_return_epo_15_weights = [0.13333, 0.03333, 0.13333, 0.03333, 0.13333, 0.03333, 0.03333, 0.03333, 0.03333, 0.03333, 0.03333, 0.13333, 0.13333, 0.03333, 0.03333]
max_return_epo_30_weights = [0.06667, 0.01667, 0.06667, 0.06667, 0.06667, 0.01667, 0.01667, 0.01667, 0.06667, 0.06667, 0.06667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667, 0.06667, 0.01667, 0.01667, 0.01667, 0.01667, 0.06667, 0.01667, 0.06667, 0.01667, 0.01667, 0.01667, 0.01667]
max_return_epo_45_weights = [0.04444, 0.04444, 0.04444, 0.04444, 0.04444, 0.04444, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.04444, 0.01111, 0.01111, 0.04444, 0.01111, 0.01111, 0.04444, 0.01111, 0.04444, 0.04444, 0.04444, 0.01111, 0.04444, 0.01111, 0.01111, 0.04444, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.04444]

max_return_epo_15 = dict(zip(tickers_15, max_return_epo_15_weights))
max_return_epo_30 = dict(zip(tickers_30, max_return_epo_30_weights))
max_return_epo_45 = dict(zip(tickers_45, max_return_epo_45_weights))

-----

## Rebalncing

In [25]:
# 분기별 (3개월) 리밸런싱
# LookBack period : 1year
# out-of-sample 기간 : 2023-05-01 ~ 2023-10-10

In [26]:
mkt = pd.read_csv('./new_market.csv')
mkt.set_index('pricingDate', inplace=True)
mkt.index = pd.to_datetime(mkt.index)
mkt_in = mkt.loc[:'2023-04-30', :]
mkt_out = mkt.loc['2023-05-01':, :]

In [27]:
# in-sample
mkt_in

,A,AAL,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
pricingDate,,,,,,,,,,,,,,,,,,,,,
2018-05-01,63.574105,42.342797,40.191708,79.940979,53.921780,26.876666,140.357973,224.08,79.992862,38.801281,...,180.343060,39.841488,58.326616,47.887170,65.827974,78.555893,108.770822,136.78,46.462606,80.580399
2018-05-02,63.257386,41.249071,41.967178,78.609543,53.032556,26.310000,139.240061,221.10,79.488744,38.284275,...,179.572242,39.747543,58.212919,47.839888,64.434146,72.716789,109.245154,133.12,46.580767,77.198727
2018-05-03,63.670080,41.122121,42.043236,78.539055,53.105894,25.983333,140.182433,226.05,79.137661,37.569084,...,180.126855,39.696300,58.015844,46.563275,65.135737,74.956321,107.124611,133.54,45.956201,77.662499
2018-05-04,64.303518,41.854527,43.692736,78.452903,53.848442,26.276666,141.762291,228.51,81.262162,37.793120,...,180.869471,39.619435,58.288717,47.272504,65.556692,74.738715,108.091876,134.67,46.808649,78.802606
2018-05-07,64.677822,41.541303,44.008850,78.029977,54.380142,26.376666,140.709052,230.99,81.469211,37.646635,...,180.399461,39.397382,58.925421,44.369392,65.500564,75.355266,108.808024,137.48,46.850849,79.372659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-24,137.937948,13.320000,164.878984,162.280931,109.873056,72.510000,275.106410,377.34,185.188487,79.896422,...,113.919233,69.814765,116.230360,41.123938,103.855012,137.610165,137.611859,288.19,27.463377,175.963019
2023-04-25,129.859693,12.840000,163.323240,163.091940,109.445108,72.590000,268.607053,369.59,179.423633,75.051218,...,111.083193,69.962199,114.578355,40.657522,102.245627,136.530286,138.528342,284.10,25.967317,172.640326
2023-04-26,132.598928,12.740000,163.313267,160.025930,108.230931,72.290000,269.113109,363.06,178.948181,76.168582,...,108.913871,68.487862,113.526185,40.617827,100.318339,136.728429,138.548265,280.42,25.637019,172.560501


In [28]:
# out-sample
mkt_out

,A,AAL,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
pricingDate,,,,,,,,,,,,,,,,,,,,,
2023-05-01,135.547342,13.89,169.127363,152.024633,110.579667,75.900000,277.259633,374.150000,180.810368,76.929971,...,115.033747,68.969479,112.759183,42.066692,103.815275,140.909242,138.299221,287.450000,26.054749,179.165975
2023-05-02,133.993448,13.77,168.080227,149.957549,110.967805,76.030000,274.600354,368.660000,181.919755,74.675468,...,112.645503,67.917785,108.265335,41.778904,103.835143,141.573020,142.463240,255.440000,23.237495,176.511812
2023-05-03,134.242470,13.79,166.993201,147.613535,111.226564,75.620000,273.052415,345.250000,180.661789,74.181059,...,108.794459,67.328050,106.131495,40.171259,103.467568,136.034930,139.086200,258.500000,22.013446,177.339990
2023-05-04,133.355954,13.51,165.337729,145.744258,110.330860,72.920000,264.439525,335.830000,179.611833,73.390006,...,108.714851,67.927614,104.272990,40.538437,105.633284,135.054123,136.107632,265.000000,19.361341,178.078366
2023-05-05,132.778224,13.87,173.096506,146.406912,110.728951,74.810000,263.596097,348.400000,181.652313,75.110547,...,110.237357,68.379744,106.868998,40.985005,108.255985,136.213258,137.382738,273.800000,23.082060,185.821338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-25,103.400002,11.04,170.874893,145.259995,93.570000,83.489998,292.679993,521.140015,159.789993,70.612259,...,88.681633,59.470001,107.606010,31.809999,87.473717,120.309998,103.639999,198.910004,29.273745,163.279068
2023-10-26,104.309998,11.15,166.670425,145.199997,93.980003,82.449997,292.040008,514.280029,160.860001,71.407326,...,87.285698,59.770000,106.624985,30.680000,88.400681,118.750000,103.120003,204.830002,30.083887,157.692444
2023-10-27,102.769997,10.92,167.998672,138.929993,92.849998,81.360001,290.040008,508.119995,160.570007,69.608482,...,87.026451,58.310001,104.593567,30.600000,87.882370,119.440002,103.190002,207.179993,29.095911,155.657318


## out-of-sample rebalancing

In [29]:
## market 데이터 불러오기
def downloads(dict, start_date, end_date):
    data = pd.DataFrame()
    a = pd.DataFrame()
    tickers = list(dict.keys())

    for ticker in tqdm(tickers):
        a = mkt.loc[start_date:end_date, ticker]
        data = pd.concat([data, a], axis=1)

    return data

In [30]:
### 예비용

# 초기 투자 금액 (1만 달러)
initial = 10000

# 초기 가중치 w* : GMW+EPO weight
## 종목 리스트
tickers_15
tickers_30
tickers_45
## 가중치 리스트
gmv_epo_15_weights
gmv_epo_30_weights
gmv_epo_45_weights
## 종목 : 가중치 딕셔너리
gmv_epo_15
gmv_epo_30
gmv_epo_45

# w* 이용하여 2023-05-01 부터 2023-07-31까지(3개월간) 투자 후 payoff(2023-07-31의 종가 * w*) 확인 
a = mkt_out.loc['2023-07-31':'2023-07-31', tickers_15] # 2023-07-31 종가 df
b = [] # 2023-07-31 종가 저장 리스트
for i in a.columns:
    b.append(float(a[i]))
# payoff_med_15 = sum(np.multiply(b, gmv_epo_15_weights)) 

# w* 이용하여 2023-05-01 당시의 초기 투자량(payoff) 확인
c = mkt_out.loc['2023-05-01':'2023-05-01', tickers_15] # 2023-05-01 종가 df
d = [] # 2023-05-01 종가 저장 리스트
for i in c.columns:
    d.append(float(c[i]))
k = list((np.divide(np.multiply(gmv_epo_15_weights, [initial] * 15), d))) # 초기 투자 주식 보유량(XX주)

# 3개월 동안의 투자 수익률 확인 (소수점 단위)
ret_3m = ((sum(np.multiply(k,b)) - initial)/initial)   # ((2023-07-31의 payoff - 초기 payoff) / 초기 payoff)

# 3개월 후 나의 총 투자 (손)수익
total_ret_3m = initial*(1+ret_3m)  # 달러

# 2022-08-01~2023-07-31까지(lookback = 1yr)의 데이터로 EPO 최적화
## 리밸런싱을 위한 GMV+EPO 최적화
def gmv_epo(dict):

    data = downloads(dict, "2022-08-01", "2023-07-31")  # mkt 데이터에서 일별 주가데이터 가져오기(lookback 1yr)
    ret = data.pct_change().dropna()
    
    num_assets = data.shape[1] # 종목 개수가 나와야함
    weights = cp.Variable((num_assets,1)) # 종목 개수랑 같아야함
    # corr_mat = np.corr(ret.values.T) # 개별 종목 별 기대수익률을 구해서 Correlation Matrix를 만듬
    corr_mat = ret.corr()

    theta = 0.75 # shrinkage parameter
    identity_matrix = np.eye(corr_mat.shape[0])  # identity matrix
    corr_mat = (1-theta) * corr_mat + theta * identity_matrix # corr_mat에 shrinkage parameter 적용

    print(corr_mat)
    
    obj = cp.Minimize(cp.quad_form(weights, corr_mat)) # 목적식 설정
    
    # 가중치 합 = 1, weights >= 0, min weight ~ max weight 범위 제약
    const = [cp.sum(weights) == 1, weights >= 0, ((1/num_assets))/2 <= weights, weights <= 2*(1/num_assets)] # 제약식 설정, 공매도 원하면 위에 주석 가져다 옆으로 복붙하면됨
    
    problem = cp.Problem(obj, const) # 문제 정의
    problem.solve(verbose=False, solver=cp.CPLEX) # 문제 풀기
    
    if problem.status == "optimal":
        w_opt = np.array(weights.value).flatten()
        print("Optimal")
        # print(gmv_weights.value[:5])
        
        # print(max(gmv_weights.value)) # 가장 큰 weight를 보고싶으면 이거 주석 해제
    else:
        print("It isn't optimal")

    return w_opt
gmv_epo_15_rebal = list(gmv_epo(stocks_15))  # 리밸런싱 시점에서의 최적 가중치 w**

# w** 이용하여 2023-08-01 부터 2023-10-10까지(3개월간) 투자 후 payoff(2023-10-10의 종가 * w**) 확인 
aa = mkt_out.loc['2023-10-10':'2023-10-10', tickers_15] # 2023-10-10 종가 df
bb = [] # 2023-10-10 종가 저장 리스트
for i in aa.columns:
    bb.append(float(aa[i]))
# payoff_final_15 = sum(np.multiply(bb, gmv_epo_15_rebal)) 

# w** 이용하여 2023-08-01 당시의 투자량(payoff) 확인
cc = mkt_out.loc['2023-08-01':'2023-08-01', tickers_15] # 2023-08-01 종가 df
dd = [] # 2023-08-01 종가 저장 리스트
for i in cc.columns:
    dd.append(float(cc[i]))
kk = list((np.divide(np.multiply(gmv_epo_15_rebal, [total_ret_3m] * 15), dd))) # 2023-08-01 기준 주식 보유량(XX주)

# 3개월 동안의 투자 수익률 확인 (소수점 단위)
ret_rebal_3m = ((sum(np.multiply(kk,bb)) - initial)/initial)   # ((2023-07-31의 payoff - 초기 payoff) / 초기 payoff)

# 2023-10-10 기준 나의 총 투자 (손)수익
total_ret_rebal_3m = initial*(1+ret_rebal_3m)  # 달러
total_ret_rebal_3m

C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\2475768508.py:24: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  b.append(float(a[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\2475768508.py:31: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  d.append(float(c[i]))
100%|██████████| 15/15 [00:00<00:00, 681.68it/s]

           AAPL      AMZN      NVDA       JPM        PG       PFE       JNJ  \
AAPL   1.000000  0.156846  0.148226  0.109627  0.124032  0.080362  0.072538   
AMZN   0.156846  1.000000  0.130132  0.086674  0.085648  0.054736  0.038545   
NVDA   0.148226  0.130132  1.000000  0.096403  0.062810  0.036387  0.019261   
JPM    0.109627  0.086674  0.096403  1.000000  0.091865  0.084914  0.067003   
PG     0.124032  0.085648  0.062810  0.091865  1.000000  0.105453  0.119241   
PFE    0.080362  0.054736  0.036387  0.084914  0.105453  1.000000  0.126516   
JNJ    0.072538  0.038545  0.019261  0.067003  0.119241  0.126516  1.000000   
KO     0.131221  0.090151  0.082269  0.104681  0.182636  0.105559  0.119147   
XOM    0.081454  0.036512  0.045085  0.103453  0.034598  0.047375  0.028131   
NEE    0.124573  0.084284  0.071742  0.083871  0.131741  0.088978  0.082797   
GOOGL  0.172059  0.174978  0.130421  0.078785  0.088158  0.066840  0.055018   
MSFT   0.179575  0.175681  0.163835  0.084578  0.108

Optimal


C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\2475768508.py:82: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  bb.append(float(aa[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\2475768508.py:89: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  dd.append(float(cc[i]))


10714.172641799314

In [31]:
ret_3m, total_ret_3m

(0.13545053919184957, 11354.505391918494)

In [32]:
ret_rebal_3m, total_ret_rebal_3m

(0.07141726417993141, 10714.172641799314)

In [33]:
# 초기 투자 금액 (1만 달러)
initial = 10000

# 초기 가중치 w* : GMW+EPO weight
## 종목 리스트
tickers_15
tickers_30
tickers_45
## 가중치 리스트
gmv_epo_15_weights
gmv_epo_30_weights
gmv_epo_45_weights
## 종목 : 가중치 딕셔너리
gmv_epo_15
gmv_epo_30
gmv_epo_45

{'AAPL': 0.011111111975729409,
 'GOOGL': 0.026646602204962858,
 'MSFT': 0.011111111299922065,
 'NVDA': 0.0309210193238956,
 'AMD': 0.025007258123960922,
 'ORCL': 0.034659892275724705,
 'CRM': 0.01111111146901072,
 'INTC': 0.011111111747740477,
 'CSCO': 0.011111111516746397,
 'JPM': 0.01779091912009698,
 'GS': 0.01984888218960698,
 'BAC': 0.02522647335013622,
 'MS': 0.011180137115527101,
 'AXP': 0.017489649124790285,
 'C': 0.028191045792364738,
 'JNJ': 0.03631158995254371,
 'UNH': 0.044444444090877805,
 'PFE': 0.016302194182152667,
 'ABBV': 0.011111111266490585,
 'TSLA': 0.011111111595258412,
 'AMGN': 0.01111111125737019,
 'GILD': 0.022827040798679366,
 'PG': 0.020587601244674014,
 'KO': 0.011111111503325005,
 'NKE': 0.011111111556128327,
 'PEP': 0.044444443639370106,
 'COST': 0.011111111461049875,
 'WMT': 0.04444444402780661,
 'TGT': 0.011111111368812045,
 'XOM': 0.011111112285545484,
 'CVX': 0.023035029376855205,
 'NEE': 0.01260693000477375,
 'DUK': 0.02165124524507929,
 'SO': 0.02165

-----------------

## 최종

## out-of-sample rebalancing : GMV + EPO

### step01 : market data import

In [34]:
## market 데이터 불러오기
def downloads(dict, start_date, end_date):
    data = pd.DataFrame()
    a = pd.DataFrame()
    tickers = list(dict.keys())

    for ticker in tqdm(tickers):
        a = mkt.loc[start_date:end_date, ticker]
        data = pd.concat([data, a], axis=1)

    return data

### step02 : 최적화 함수 적용

#### GMV + EPO

In [35]:
## 리밸런싱을 위한 GMV+EPO 최적화
def gmv_epo(dict, end, lookback_yr):
            # 종목:가중치 딕셔너리, lookback 기간의 마지막 날짜 'YYYY-MM-DD', lookback 기간(int)

    before = pd.to_datetime(end) - pd.DateOffset(years=lookback_yr) + pd.DateOffset(days=1)
    data = downloads(dict, before, end)  # mkt 데이터에서 일별 주가데이터 가져오기(lookback 기간만큼)
    ret = data.pct_change().dropna()
    
    num_assets = data.shape[1] # 종목 개수가 나와야함
    weights = cp.Variable((num_assets,1)) # 종목 개수랑 같아야함
    # corr_mat = np.corr(ret.values.T) # 개별 종목 별 기대수익률을 구해서 Correlation Matrix를 만듬
    corr_mat = ret.corr()

    theta = 0.75 # shrinkage parameter
    identity_matrix = np.eye(corr_mat.shape[0])  # identity matrix
    corr_mat = (1-theta) * corr_mat + theta * identity_matrix # corr_mat에 shrinkage parameter 적용

    # print(corr_mat)
    
    obj = cp.Minimize(cp.quad_form(weights, corr_mat)) # 목적식 설정
    
    # 가중치 합 = 1, weights >= 0, min weight ~ max weight 범위 제약
    const = [cp.sum(weights) == 1, weights >= 0, ((1/num_assets))/2 <= weights, weights <= 2*(1/num_assets)] # 제약식 설정, 공매도 원하면 위에 주석 가져다 옆으로 복붙하면됨
    
    problem = cp.Problem(obj, const) # 문제 정의
    problem.solve(verbose=False, solver=cp.CPLEX) # 문제 풀기
    
    if problem.status == "optimal":
        w_opt = np.array(weights.value).flatten()
        print("Optimal")
        # print(gmv_weights.value[:5])
        
        # print(max(gmv_weights.value)) # 가장 큰 weight를 보고싶으면 이거 주석 해제
    else:
        print("It isn't optimal")

    return w_opt

### step03 : rebalancing

In [54]:
# start부터 3개월간 투자 시, 수익률과 (손)수익 확인하는 함수
# parameter: 시작시점, 끝시점, 종목 딕서녀리, 최적 종목 들어있는 리스트, lookback 기간(int)
def my_return01(start, end, stock_dict, ticker_lst, lookback):

    ret_rebal = []  # 리밸런싱 시점에서의 수익률(소수점 단위)
    money_rebal = []  # 리밸런싱 시점에서의 (손)수익
    initial = 10000  # 초기 투자 금액 (1만 달러)
    w = []
      # 시작 시점 기준 3개월 뒤 시점
    dates = pd.date_range(start, end, freq='3MS')
    # ['2023-05-01', '2023-08-01']

    for i in range(len(dates)):
        start = dates[i]
        rebal = start + pd.DateOffset(months=3) - pd.DateOffset(days=1)
        # w* 이용하여 start 시점부터 3개월간 투자할 때, 3개월 뒤의 종가 확인
        price_3m = mkt_out.loc[rebal:, ticker_lst][0:1]  # 3개월 뒤 시점의 종가 df
        lst_3m = [] # 3개월 뒤 시점의 종가 저장 리스트
        for i in price_3m.columns:
            lst_3m.append(float(price_3m[i]))

        # w* 이용하여 start 시점 당시의 초기 투자량 확인
        price_now = mkt_out.loc[start:, ticker_lst][0:1]  # start 시점의 종가 df
        lst_now = []  # start 시점의 종가 저장 리스트
        for j in price_now.columns:
            lst_now.append(float(price_now[j]))

        # 리밸런싱을 위한 GMV + EPO 최적화
        opt_w = list(gmv_epo(stock_dict, start, lookback))  # 리밸런싱 시점에서의 최적 가중치 w**
        w.append(opt_w)

        # 초기 투자 주식 보유량 (XX주)
        a = np.multiply(opt_w, ([initial] * len(ticker_lst) ) ) 
        count_stocks = np.divide(a, np.array(lst_now))

        # 3개월 동안의 투자 수익률 확인 (소수점 단위) : (3개월 뒤 시점의 payoff - start 시점의 payoff) / start 시점의 payoff
        b = sum(np.multiply(count_stocks, np.array(lst_3m)))
        ret_3m = (b - initial) / initial
        ret_rebal.append(ret_3m)

        # 3개월 후 총 투자 (손)수익 (달러)
        total_ret_3m = initial * (1 + ret_3m)
        initial = total_ret_3m
        money_rebal.append(total_ret_3m)

    return ret_rebal, money_rebal, w

-----

### step04 : results
- 리밸런싱 시점에서의 총 (손)수익, 리밸런싱 시점에서의 (손)수익률(소수점 단위)
- 마지막 시점에서의 총 (손)수익, 마지막 시점에서의 (손)수익률(소수점 단위)

In [56]:
# 15종목
ret, money, w = my_return01('2023-05-01', '2023-10-01', stocks_15, tickers_15, 1)

print('2023-05-01 시점의 최적 가중치 :', w[0])
print('2023-08-01 시점의 최적 가중치 :', w[1])
print('-------------------------------------------------------')
print('리밸런싱 시점에서의 (손)수익률(소수점 단위) :', ret[0])
print('리밸런싱 시점에서의 (손)수익 : $', money[0])

print()
print('마지막 시점에서의 총 (손)수익률(소수점 단위) :', ret[1])
print('마지막 시점에서의 총 (손)수익 : $', money[1])


C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\1171885175.py:20: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_3m.append(float(price_3m[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\1171885175.py:26: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_now.append(float(price_now[j]))
100%|██████████| 15/15 [00:00<00:00, 1110.43it/s]
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\1171885175.py:20: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_3m.append(float(price_3m[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\1171885175.py:26: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use flo

Optimal


100%|██████████| 15/15 [00:00<00:00, 1249.72it/s]


Optimal
2023-05-01 시점의 최적 가중치 : [0.03416654462460771, 0.05946767330970594, 0.05862942026389745, 0.060419137930808504, 0.07806369589736416, 0.08027148789655082, 0.089710790745334, 0.061116112303139625, 0.10640430221706078, 0.06928004925994374, 0.05537463188618921, 0.04628140611140862, 0.08704906321556695, 0.053599434588976355, 0.06016624974945417]
2023-08-01 시점의 최적 가중치 : [0.0334842471227102, 0.06008824965292256, 0.06961870596860825, 0.058878937103498494, 0.06351287934652342, 0.08131443519736826, 0.09447777791698607, 0.052488593213472615, 0.10199583877924796, 0.0670461883204847, 0.058617968243817396, 0.05034426275911745, 0.09152805037450878, 0.05716103582968125, 0.05944283017122806]
-------------------------------------------------------
리밸런싱 시점에서의 (손)수익률(소수점 단위) : 0.12966111414241532
리밸런싱 시점에서의 (손)수익 : $ 11296.611141424153

마지막 시점에서의 총 (손)수익률(소수점 단위) : -0.09998895991314333
마지막 시점에서의 총 (손)수익 : $ 10167.074742849925


In [57]:
# 30종목
ret, money, w = my_return01('2023-05-01', '2023-10-01', stocks_30, tickers_30, 1)

print('2023-05-01 시점의 최적 가중치 :', w[0])
print('2023-08-01 시점의 최적 가중치 :', w[1])
print('-------------------------------------------------------')
print('리밸런싱 시점에서의 (손)수익률(소수점 단위) :', ret[0])
print('리밸런싱 시점에서의 (손)수익 : $', money[0])

print()
print('마지막 시점에서의 총 (손)수익률(소수점 단위) :', ret[1])
print('마지막 시점에서의 총 (손)수익 : $', money[1])

C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\1171885175.py:20: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_3m.append(float(price_3m[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\1171885175.py:26: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_now.append(float(price_now[j]))
100%|██████████| 30/30 [00:00<00:00, 1153.62it/s]
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\1171885175.py:20: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_3m.append(float(price_3m[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\1171885175.py:26: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use flo

Optimal


100%|██████████| 30/30 [00:00<00:00, 1304.00it/s]


Optimal
2023-05-01 시점의 최적 가중치 : [0.016666693675091663, 0.0302190890909724, 0.026180741512881144, 0.016666668911905436, 0.022701054717866348, 0.024607998460271646, 0.04610974877169958, 0.03955655497346591, 0.06575480834368308, 0.020176089586109825, 0.03796881105813261, 0.02032182404207065, 0.02117719829860075, 0.019817060242874312, 0.03325077952558616, 0.047053893091651866, 0.05107300968645141, 0.04141626257753937, 0.056682886618092905, 0.0319399813109354, 0.018776176259472668, 0.05833804134605087, 0.032046379958936624, 0.04178008849204673, 0.06405658246808965, 0.024027852698789202, 0.019205730518267922, 0.016666670031039117, 0.03021647151629577, 0.02554485221513026]
2023-08-01 시점의 최적 가중치 : [0.020011607367278396, 0.033119986187056236, 0.0406478959357913, 0.01666666824840471, 0.030796804816913728, 0.02649989745461069, 0.050105680427987354, 0.04085442521415764, 0.06310401655303899, 0.02407925997416787, 0.02268603424445393, 0.016666673414874985, 0.020173934453576733, 0.020554064123476868, 

In [58]:
# 45종목
ret, money, w = my_return01('2023-05-01', '2023-10-01', stocks_45, tickers_45, 1)

print('2023-05-01 시점의 최적 가중치 :', w[0])
print('2023-08-01 시점의 최적 가중치 :', w[1])
print('-------------------------------------------------------')
print('리밸런싱 시점에서의 (손)수익률(소수점 단위) :', ret[0])
print('리밸런싱 시점에서의 (손)수익 : $', money[0])

print()
print('마지막 시점에서의 총 (손)수익률(소수점 단위) :', ret[1])
print('마지막 시점에서의 총 (손)수익 : $', money[1])

C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\1171885175.py:20: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_3m.append(float(price_3m[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\1171885175.py:26: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_now.append(float(price_now[j]))
100%|██████████| 45/45 [00:00<00:00, 1071.20it/s]


C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\1171885175.py:20: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_3m.append(float(price_3m[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\1171885175.py:26: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_now.append(float(price_now[j]))


Optimal


100%|██████████| 45/45 [00:00<00:00, 1363.33it/s]


Optimal
2023-05-01 시점의 최적 가중치 : [0.01111111126116994, 0.013073479572892253, 0.011111111741805327, 0.018040862509462127, 0.01877042715922211, 0.019241379769647418, 0.022815671823744617, 0.011823270176486565, 0.014967239425840512, 0.01111111176171712, 0.011111111636325471, 0.01111111167015062, 0.011111111377814415, 0.011111111995038525, 0.011112697093206498, 0.027886228480870884, 0.02885926514562019, 0.02161413860523313, 0.044444444028454495, 0.04444444330608938, 0.04033051590591301, 0.03301927619673047, 0.021052182688287655, 0.011111111533860853, 0.01111111161898528, 0.01133929543641628, 0.011111114000701779, 0.04444444323873353, 0.024894760576679664, 0.037209885676189866, 0.03292240533054593, 0.0187276907924751, 0.02474235330022852, 0.020061149022054057, 0.044444443584470895, 0.0177251840850195, 0.01923869104370415, 0.011111111261945127, 0.017202332824835947, 0.011111111246301469, 0.01954373085826516, 0.0426748122923424, 0.011111111974307475, 0.044444403317756856, 0.04444444365244824]


----

## out-of-sample rebalancing : Max Sharpe + EPO

### step01 : market data import

In [59]:
## market 데이터 불러오기
def downloads(dict, start_date, end_date):
    data = pd.DataFrame()
    a = pd.DataFrame()
    tickers = list(dict.keys())

    for ticker in tqdm(tickers):
        a = mkt.loc[start_date:end_date, ticker]
        data = pd.concat([data, a], axis=1)

    return data

### step02 : 최적화 함수 적용

#### Max Sharpe + EPO

In [60]:
# Max Sharpe 함수 + shrinkage parameter
def max_sharpe_epo(dict, end, lookback_yr):
    # 종목:가중치 딕셔너리, lookback 기간의 마지막 날짜 'YYYY-MM-DD', lookback
    
#     df = downloads(dict, "2018-05-01", "2023-04-30")
    before = pd.to_datetime(end) - pd.DateOffset(years=lookback_yr) + pd.DateOffset(days=1)
    df = downloads(dict, before, end)  # mkt 데이터에서 일별 주가데이터 가져오기(lookback 기간만큼)
#     ret = data.pct_change().dropna()

    mu = expected_returns.mean_historical_return(df) # pfo mean 
    S = risk_models.sample_cov(df) # cov
    stds = np.sqrt(np.diag(S)) # cov의 표준편차
    S = risk_models.cov_to_corr(S) # cov -> corr
    theta = 0.75 # shrinkage parameter
    identity_matrix = np.eye(S.shape[0])  # identity matrix
    S = (1-theta)* S + theta * identity_matrix # shrinkage 반영된 correlation matrix
    S = risk_models.corr_to_cov(S, stds)
    ef = EfficientFrontier(mu, S)

    n_assets = len(mu)  # 자산 수
    min_weights = 0.5 * 1 / n_assets  # 각 자산별 최소 weight
    max_weights = 2 * 1 / n_assets

    for asset in mu.index:
        num = mu.index.get_loc(asset)
        ef.add_constraint(lambda w: w[num] <= max_weights)
        ef.add_constraint(lambda w: w[num] >= min_weights)
    
    ef.add_constraint(lambda w: w.sum() == 1)

    w_opt = ef.max_sharpe() # Max Sharpe ratio가 되는 weights 찾기
    # print('w_opt:', w_opt)
    w_clean = ef.clean_weights()
    # print('w_clean',w_clean.values())
    ef.portfolio_performance(verbose=True) 
    # print(w_clean) # 가중치 확인
    max_weight_asset = max(w_clean, key=w_clean.get)
    max_weight_value = w_clean[max_weight_asset]

#     print()
#     print("가장 큰 가중치를 가진 자산:", max_weight_asset)
#     print("가장 큰 가중치:", max_weight_value)

    return w_clean  #.values()

### step03 : rebalancing

In [63]:
# start부터 3개월간 투자 시, 수익률과 (손)수익 확인하는 함수
# parameter: 시작시점, 끝시점, 종목 딕셔너리, 최적 종목 들어있는 리스트, lookback 기간(int)
def my_return02(start, end, stock_dict, ticker_lst, lookback):

    ret_rebal = []  # 리밸런싱 시점에서의 수익률(소수점 단위)
    money_rebal = []  # 리밸런싱 시점에서의 (손)수익
    initial = 10000  # 초기 투자 금액 (1만 달러)
    w = []

      # 시작 시점 기준 3개월 뒤 시점
    dates = pd.date_range(start, end, freq='3MS')
    # ['2023-05-01', '2023-08-01']

    for i in range(len(dates)):
        start = dates[i]
        rebal = start + pd.DateOffset(months=3) - pd.DateOffset(days=1)
        # w* 이용하여 start 시점부터 3개월간 투자할 때, 3개월 뒤의 종가 확인
        price_3m = mkt_out.loc[rebal:, ticker_lst][0:1]  # 3개월 뒤 시점의 종가 df
        lst_3m = [] # 3개월 뒤 시점의 종가 저장 리스트
        for i in price_3m.columns:
            lst_3m.append(float(price_3m[i]))

        # w* 이용하여 start 시점 당시의 초기 투자량 확인
        price_now = mkt_out.loc[start:, ticker_lst][0:1]  # start 시점의 종가 df
        lst_now = []  # start 시점의 종가 저장 리스트
        for j in price_now.columns:
            lst_now.append(float(price_now[j]))

        # 리밸런싱을 위한 Max Sharpe + EPO 최적화
        opt_w = list(max_sharpe_epo(stock_dict, start, lookback).values())  # 리밸런싱 시점에서의 최적 가중치 w**
        w.append(opt_w)

        # 초기 투자 주식 보유량 (XX주)
        a = np.multiply(opt_w, ([initial] * len(ticker_lst) ) ) 
        count_stocks = np.divide(a, np.array(lst_now))

        # 3개월 동안의 투자 수익률 확인 (소수점 단위) : (3개월 뒤 시점의 payoff - start 시점의 payoff) / start 시점의 payoff
        b = sum(np.multiply(count_stocks, np.array(lst_3m)))
        ret_3m = (b - initial) / initial
        ret_rebal.append(ret_3m)

        # 3개월 후 총 투자 (손)수익 (달러)
        total_ret_3m = initial * (1 + ret_3m)
        initial = total_ret_3m
        money_rebal.append(total_ret_3m)

    return ret_rebal, money_rebal, w

### step04 : results
- 리밸런싱 시점에서의 총 (손)수익, 리밸런싱 시점에서의 (손)수익률(소수점 단위)
- 마지막 시점에서의 총 (손)수익, 마지막 시점에서의 (손)수익률(소수점 단위)

In [64]:
# 15종목
ret, money, w = my_return02('2023-05-01', '2023-10-01', stocks_15, tickers_15, 1)

print('2023-05-01 시점의 최적 가중치 :', w[0])
print('2023-08-01 시점의 최적 가중치 :', w[1])
print('-------------------------------------------------------')
print('리밸런싱 시점에서의 (손)수익률(소수점 단위) :', ret[0])
print('리밸런싱 시점에서의 (손)수익 : $', money[0])

print()
print('마지막 시점에서의 총 (손)수익률(소수점 단위) :', ret[1])
print('마지막 시점에서의 총 (손)수익 : $', money[1])

C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\3741201858.py:21: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_3m.append(float(price_3m[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\3741201858.py:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_now.append(float(price_now[j]))
100%|██████████| 15/15 [00:00<00:00, 1250.69it/s]
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\3741201858.py:21: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_3m.append(float(price_3m[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\3741201858.py:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use flo

Expected annual return: 13.6%
Annual volatility: 15.6%
Sharpe Ratio: 0.74


100%|██████████| 15/15 [00:00<00:00, 1363.29it/s]

Expected annual return: 32.7%
Annual volatility: 13.9%
Sharpe Ratio: 2.20
2023-05-01 시점의 최적 가중치 : [0.03333, 0.03333, 0.13334, 0.13333, 0.03333, 0.03333, 0.03333, 0.13333, 0.13334, 0.13333, 0.03333, 0.03333, 0.03333, 0.03333, 0.03333]
2023-08-01 시점의 최적 가중치 : [0.09144, 0.03333, 0.13333, 0.13333, 0.13333, 0.03333, 0.04546, 0.03333, 0.1208, 0.03333, 0.03333, 0.07564, 0.03333, 0.03333, 0.03333]
-------------------------------------------------------
리밸런싱 시점에서의 (손)수익률(소수점 단위) : 0.12943882716590488
리밸런싱 시점에서의 (손)수익 : $ 11294.388271659049

마지막 시점에서의 총 (손)수익률(소수점 단위) : -0.08462420191952638
마지막 시점에서의 총 (손)수익 : $ 10338.609678000643


In [65]:
# 30종목
ret, money, w = my_return02('2023-05-01', '2023-10-01', stocks_30, tickers_30, 1)

print('2023-05-01 시점의 최적 가중치 :', w[0])
print('2023-08-01 시점의 최적 가중치 :', w[1])
print('-------------------------------------------------------')
print('리밸런싱 시점에서의 (손)수익률(소수점 단위) :', ret[0])
print('리밸런싱 시점에서의 (손)수익 : $', money[0])

print()
print('마지막 시점에서의 총 (손)수익률(소수점 단위) :', ret[1])
print('마지막 시점에서의 총 (손)수익 : $', money[1])

C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\3741201858.py:21: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_3m.append(float(price_3m[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\3741201858.py:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_now.append(float(price_now[j]))
100%|██████████| 30/30 [00:00<00:00, 1303.98it/s]
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\3741201858.py:21: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_3m.append(float(price_3m[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\3741201858.py:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use flo

Expected annual return: 14.3%
Annual volatility: 12.5%
Sharpe Ratio: 0.98


100%|██████████| 30/30 [00:00<00:00, 1199.71it/s]

Expected annual return: 28.8%
Annual volatility: 10.6%
Sharpe Ratio: 2.53
2023-05-01 시점의 최적 가중치 : [0.01667, 0.01667, 0.06667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667, 0.06667, 0.01667, 0.01667, 0.04862, 0.06667, 0.06667, 0.01667, 0.03471, 0.06667, 0.06667, 0.01667, 0.06667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667, 0.06667, 0.06667, 0.01667]
2023-08-01 시점의 최적 가중치 : [0.02417, 0.01667, 0.06667, 0.02454, 0.01667, 0.03162, 0.01667, 0.01667, 0.06667, 0.01667, 0.06667, 0.01667, 0.06667, 0.01667, 0.06667, 0.01667, 0.04213, 0.06667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667, 0.06667, 0.01667, 0.01667, 0.06088, 0.06667, 0.01667]
-------------------------------------------------------
리밸런싱 시점에서의 (손)수익률(소수점 단위) : 0.09486454807814389
리밸런싱 시점에서의 (손)수익 : $ 10948.645480781439

마지막 시점에서의 총 (손)수익률(소수점 단위) : -0.08357952700031049
마지막 시점에서의 총 (손)수익 : $ 10033.56287020364


In [66]:
# 45종목
ret, money, w = my_return02('2023-05-01', '2023-10-01', stocks_45, tickers_45, 1)

print('2023-05-01 시점의 최적 가중치 :', w[0])
print('2023-08-01 시점의 최적 가중치 :', w[1])
print('-------------------------------------------------------')
print('리밸런싱 시점에서의 (손)수익률(소수점 단위) :', ret[0])
print('리밸런싱 시점에서의 (손)수익 : $', money[0])

print()
print('마지막 시점에서의 총 (손)수익률(소수점 단위) :', ret[1])
print('마지막 시점에서의 총 (손)수익 : $', money[1])

C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\3741201858.py:21: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_3m.append(float(price_3m[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\3741201858.py:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_now.append(float(price_now[j]))
100%|██████████| 45/45 [00:00<00:00, 1363.30it/s]
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\3741201858.py:21: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_3m.append(float(price_3m[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\3741201858.py:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use flo

Expected annual return: 15.6%
Annual volatility: 11.0%
Sharpe Ratio: 1.23


100%|██████████| 45/45 [00:00<00:00, 1323.24it/s]


Expected annual return: 32.7%
Annual volatility: 9.9%
Sharpe Ratio: 3.10
2023-05-01 시점의 최적 가중치 : [0.01111, 0.01111, 0.01111, 0.04444, 0.01111, 0.04444, 0.01111, 0.01111, 0.01111, 0.04444, 0.04444, 0.01111, 0.02356, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.04444, 0.01111, 0.04444, 0.04444, 0.01111, 0.03625, 0.01111, 0.04444, 0.01111, 0.01111, 0.01111, 0.04444, 0.01605, 0.04444, 0.01111, 0.03525, 0.04444, 0.01111, 0.01111, 0.01111, 0.04444, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.04444]
2023-08-01 시점의 최적 가중치 : [0.01465, 0.01111, 0.01679, 0.04444, 0.01111, 0.04444, 0.01111, 0.01111, 0.04444, 0.04444, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01229, 0.01111, 0.01111, 0.04444, 0.01111, 0.01111, 0.04444, 0.04444, 0.01111, 0.01111, 0.04444, 0.01111, 0.04444, 0.01111, 0.03404, 0.01111, 0.01111, 0.01111, 0.01111, 0.04444, 0.01111, 0.04444, 0.01111, 0.04444, 0.01111, 0.01111, 0.04444, 0.01111, 0.01111, 0.04444]
-------------------------------------------------------
리밸런싱 시점에서의 

----

## out-of-sample rebalancing : Max Return + EPO

### step01 : market data import

In [46]:
## market 데이터 불러오기
def downloads(dict, start_date, end_date):
    data = pd.DataFrame()
    a = pd.DataFrame()
    tickers = list(dict.keys())

    for ticker in tqdm(tickers):
        a = mkt.loc[start_date:end_date, ticker]
        data = pd.concat([data, a], axis=1)

    return data

### step02 : 최적화 함수 적용

#### Max Return + EPO

In [47]:
def max_return_epo(dict, end, lookback_yr):
    df = downloads(dict, "2018-05-01", "2023-04-30")
    # 종목:가중치 딕셔너리, lookback 기간의 마지막 날짜 'YYYY-MM-DD', lookback
    
    before = pd.to_datetime(end) - pd.DateOffset(years=lookback_yr) + pd.DateOffset(days=1)
    df = downloads(dict, before, end)  # mkt 데이터에서 일별 주가데이터 가져오기(lookback 기간만큼)

    mu = expected_returns.mean_historical_return(df) # pfo mean 
    S = risk_models.sample_cov(df) # cov
    S = risk_models.cov_to_corr(S) # cov -> corr
    stds = np.sqrt(np.diag(S))
    theta = 0.75 # shrinkage parameter
    identity_matrix = np.eye(S.shape[0])  # identity matrix
    S = (1-theta)* S + theta * identity_matrix
    S = risk_models.corr_to_cov(S, stds)
    ef = EfficientFrontier(mu, S, solver='ECOS')
    ef.add_constraint(lambda x: x.sum() == 1,)

    n_assets = len(mu)  # 자산 수
    min_weights = 0.5 * 1 / n_assets  # 각 자산별 최소 weight
    max_weights = 2 * 1 / n_assets
    
    for asset in mu.index:
        num = mu.index.get_loc(asset)
        ef.add_constraint(lambda w: w[num] <= max_weights)
        ef.add_constraint(lambda w: w[num] >= min_weights)

    w_opt = ef._max_return()

    w_clean = ef.clean_weights()
#     print()
    ef.portfolio_performance(verbose=True) 
    # print(w_clean) # 가중치 확인
    max_weight_asset = max(w_clean, key=w_clean.get)
    max_weight_value = w_clean[max_weight_asset]

#     print()
#     print("가장 큰 가중치를 가진 자산:", max_weight_asset)
#     print("가장 큰 가중치:", max_weight_value)

    return w_clean

### step03 : rebalancing

In [67]:
# start부터 3개월간 투자 시, 수익률과 (손)수익 확인하는 함수
# parameter: 시작시점, 끝시점, 종목 딕셔너리, 최적 종목 들어있는 리스트, lookback 기간(int)
def my_return03(start, end, stock_dict, ticker_lst, lookback):

    ret_rebal = []  # 리밸런싱 시점에서의 수익률(소수점 단위)
    money_rebal = []  # 리밸런싱 시점에서의 (손)수익
    initial = 10000  # 초기 투자 금액 (1만 달러)
    w = []

      # 시작 시점 기준 3개월 뒤 시점
    dates = pd.date_range(start, end, freq='3MS')
    # ['2023-05-01', '2023-08-01']

    for i in range(len(dates)):
        start = dates[i]
        rebal = start + pd.DateOffset(months=3) - pd.DateOffset(days=1)
        # w* 이용하여 start 시점부터 3개월간 투자할 때, 3개월 뒤의 종가 확인
        price_3m = mkt_out.loc[rebal:, ticker_lst][0:1]  # 3개월 뒤 시점의 종가 df
        lst_3m = [] # 3개월 뒤 시점의 종가 저장 리스트
        for i in price_3m.columns:
            lst_3m.append(float(price_3m[i]))

        # w* 이용하여 start 시점 당시의 초기 투자량 확인
        price_now = mkt_out.loc[start:, ticker_lst][0:1]  # start 시점의 종가 df
        lst_now = []  # start 시점의 종가 저장 리스트
        for j in price_now.columns:
            lst_now.append(float(price_now[j]))

        # 리밸런싱을 위한 Max Return + EPO 최적화
        opt_w = list(max_return_epo(stock_dict, start, lookback).values())  # 리밸런싱 시점에서의 최적 가중치 w**
        w.append(opt_w)

        # 초기 투자 주식 보유량 (XX주)
        a = np.multiply(opt_w, ([initial] * len(ticker_lst) ) ) 
        count_stocks = np.divide(a, np.array(lst_now))

        # 3개월 동안의 투자 수익률 확인 (소수점 단위) : (3개월 뒤 시점의 payoff - start 시점의 payoff) / start 시점의 payoff
        b = sum(np.multiply(count_stocks, np.array(lst_3m)))
        ret_3m = (b - initial) / initial
        ret_rebal.append(ret_3m)

        # 3개월 후 총 투자 (손)수익 (달러)
        total_ret_3m = initial * (1 + ret_3m)
        initial = total_ret_3m
        money_rebal.append(total_ret_3m)

    return ret_rebal, money_rebal, w

### step04 : results
- 리밸런싱 시점에서의 총 (손)수익, 리밸런싱 시점에서의 (손)수익률(소수점 단위)
- 마지막 시점에서의 총 (손)수익, 마지막 시점에서의 (손)수익률(소수점 단위)

In [68]:
# 15종목
ret, money, w = my_return03('2023-05-01', '2023-10-01', stocks_15, tickers_15, 1)

print('2023-05-01 시점의 최적 가중치 :', w[0])
print('2023-08-01 시점의 최적 가중치 :', w[1])
print('-------------------------------------------------------')
print('리밸런싱 시점에서의 (손)수익률(소수점 단위) :', ret[0])
print('리밸런싱 시점에서의 (손)수익 : $', money[0])

print()
print('마지막 시점에서의 총 (손)수익률(소수점 단위) :', ret[1])
print('마지막 시점에서의 총 (손)수익 : $', money[1])

C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\3213127870.py:21: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_3m.append(float(price_3m[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\3213127870.py:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_now.append(float(price_now[j]))
100%|██████████| 15/15 [00:00<00:00, 1363.35it/s]
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\3213127870.py:21: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_3m.append(float(price_3m[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\3213127870.py:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use flo

Expected annual return: 14.0%
Annual volatility: 44.5%
Sharpe Ratio: 0.27


100%|██████████| 15/15 [00:00<00:00, 1363.29it/s]

Expected annual return: 34.0%
Annual volatility: 43.4%
Sharpe Ratio: 0.74
2023-05-01 시점의 최적 가중치 : [0.03333, 0.03333, 0.13333, 0.13333, 0.03333, 0.03333, 0.03333, 0.03333, 0.13333, 0.13333, 0.03333, 0.13333, 0.03333, 0.03333, 0.03333]
2023-08-01 시점의 최적 가중치 : [0.13333, 0.03333, 0.13333, 0.13333, 0.03333, 0.03333, 0.03333, 0.03333, 0.13333, 0.03333, 0.03333, 0.13333, 0.03333, 0.03333, 0.03333]
-------------------------------------------------------
리밸런싱 시점에서의 (손)수익률(소수점 단위) : 0.14254174123748745
리밸런싱 시점에서의 (손)수익 : $ 11425.417412374874

마지막 시점에서의 총 (손)수익률(소수점 단위) : -0.08429045050032868
마지막 시점에서의 총 (손)수익 : $ 10462.363831531497


In [69]:
# 30종목
ret, money, w = my_return03('2023-05-01', '2023-10-01', stocks_30, tickers_30, 1)

print('2023-05-01 시점의 최적 가중치 :', w[0])
print('2023-08-01 시점의 최적 가중치 :', w[1])
print('-------------------------------------------------------')
print('리밸런싱 시점에서의 (손)수익률(소수점 단위) :', ret[0])
print('리밸런싱 시점에서의 (손)수익 : $', money[0])

print()
print('마지막 시점에서의 총 (손)수익률(소수점 단위) :', ret[1])
print('마지막 시점에서의 총 (손)수익 : $', money[1])

C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\3213127870.py:21: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_3m.append(float(price_3m[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\3213127870.py:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_now.append(float(price_now[j]))
100%|██████████| 30/30 [00:00<00:00, 1363.28it/s]
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\3213127870.py:21: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_3m.append(float(price_3m[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\3213127870.py:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use flo

Expected annual return: 14.6%
Annual volatility: 38.5%
Sharpe Ratio: 0.33


100%|██████████| 30/30 [00:00<00:00, 1304.01it/s]


Expected annual return: 30.5%
Annual volatility: 36.9%
Sharpe Ratio: 0.77
2023-05-01 시점의 최적 가중치 : [0.01667, 0.01667, 0.06667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667, 0.06667, 0.01667, 0.01667, 0.01667, 0.06667, 0.06667, 0.01667, 0.01667, 0.06667, 0.06667, 0.01667, 0.06667, 0.01667, 0.01667, 0.06667, 0.01667, 0.01667, 0.01667, 0.01667, 0.06667, 0.06667, 0.01667]
2023-08-01 시점의 최적 가중치 : [0.06667, 0.01667, 0.06667, 0.06667, 0.01667, 0.06667, 0.01667, 0.01667, 0.06667, 0.01667, 0.01667, 0.01667, 0.06667, 0.01667, 0.06667, 0.01667, 0.01667, 0.06667, 0.01667, 0.01667, 0.01667, 0.01667, 0.06667, 0.01667, 0.06667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667]
-------------------------------------------------------
리밸런싱 시점에서의 (손)수익률(소수점 단위) : 0.1029121487581955
리밸런싱 시점에서의 (손)수익 : $ 11029.121487581955

마지막 시점에서의 총 (손)수익률(소수점 단위) : -0.08720958770690226
마지막 시점에서의 총 (손)수익 : $ 10067.276349880596


In [70]:
# 45종목
ret, money, w = my_return03('2023-05-01', '2023-10-01', stocks_45, tickers_45, 1)

print('2023-05-01 시점의 최적 가중치 :', w[0])
print('2023-08-01 시점의 최적 가중치 :', w[1])
print('-------------------------------------------------------')
print('리밸런싱 시점에서의 (손)수익률(소수점 단위) :', ret[0])
print('리밸런싱 시점에서의 (손)수익 : $', money[0])

print()
print('마지막 시점에서의 총 (손)수익률(소수점 단위) :', ret[1])
print('마지막 시점에서의 총 (손)수익 : $', money[1])

C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\3213127870.py:21: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_3m.append(float(price_3m[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\3213127870.py:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_now.append(float(price_now[j]))
100%|██████████| 45/45 [00:00<00:00, 1363.29it/s]
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\3213127870.py:21: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_3m.append(float(price_3m[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_14416\3213127870.py:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use flo

Expected annual return: 16.2%
Annual volatility: 36.1%
Sharpe Ratio: 0.39


100%|██████████| 45/45 [00:00<00:00, 1323.20it/s]


Expected annual return: 34.0%
Annual volatility: 34.4%
Sharpe Ratio: 0.93
2023-05-01 시점의 최적 가중치 : [0.01111, 0.01111, 0.04444, 0.04444, 0.01111, 0.04444, 0.04444, 0.01111, 0.01111, 0.04444, 0.04444, 0.01111, 0.04444, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.04444, 0.01111, 0.01111, 0.01111, 0.04444, 0.01111, 0.01111, 0.01111, 0.04444, 0.01111, 0.04444, 0.01111, 0.01111, 0.04444, 0.01111, 0.01111, 0.01111, 0.04444, 0.01111, 0.01111, 0.04444, 0.01111, 0.01111, 0.04444]
2023-08-01 시점의 최적 가중치 : [0.04444, 0.01111, 0.04444, 0.04444, 0.04444, 0.04444, 0.04444, 0.01111, 0.04444, 0.04444, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.04444, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.04444, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.04444, 0.01111, 0.04444, 0.01111, 0.04444, 0.01111, 0.01111, 0.04444, 0.01111, 0.01111, 0.04444]
-------------------------------------------------------
리밸런싱 시점에서의